In [ ]:
#ライブラリ読み込み
#データ読み込み
import numpy as np
import pandas as pd
import lightgbm as lgb
df = pd.read_csv("materialdata_lastver.csv", encoding = "utf-8", low_memory = False)
pd.set_option('display.max_columns',200)
pd.set_option("display.max_rows", 200)
from sklearn.ensemble import BaggingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_validate, cross_val_predict, KFold
from numba import njit

import matplotlib.pyplot as plt
import seaborn as sns

import pydot
import graphviz

import pickle

#スペースアリはYとS
#大文字被りがLI
#原子の数が0になっているのが複数あったので消去
#Yoという謎原子があったので消去
#O6050という異常値があるがデータ生成の段階では影響がないと判断
#Nd185があったのでNd1.85に訂正(周囲にNd1.85のデータがあったため)
#O1013があったのでO10.13に訂正(周囲にO10.XXのデータがあったため)
#Cu285があったのでCu2.85に訂正(周囲にCu2.85のデータがあったため)

In [ ]:
display(df)

In [ ]:
df_tcshaped = df.dropna(subset = ["tc"], axis = "index").dropna(subset = ["str3"], axis = "index").reset_index(drop = True)

In [ ]:
print(df_tcshaped["year"].isnull().sum())

In [ ]:
print(df_tcshaped["tc"].isnull().sum())

In [ ]:
X_tcshaped = pd.concat([df_tcshaped.loc[:, "ma1":"mo1"], df_tcshaped["str3"]], axis = "columns")

display(X_tcshaped)

In [ ]:
y = df_tcshaped["tc"].astype(float)

In [ ]:
y = pd.DataFrame(y)
y["tc_pred"] = None

display(y)

In [ ]:
#ダミー変数の作成今回は結晶構造を変更
encoder = LabelEncoder()
array_dummiedcrystal = encoder.fit_transform(X_tcshaped["str3"])

In [ ]:
df_dummiedcrystal = pd.DataFrame(array_dummiedcrystal, columns = ["name of structure"])

In [ ]:
X_withstr = pd.concat([X_tcshaped, df_dummiedcrystal], axis = "columns").drop(["str3"], axis = "columns")

In [ ]:
df_oxygendummies = pd.get_dummies(X_withstr["mo1"])
df_oxygendummies = df_oxygendummies.rename(columns = {"O" : "Oarinashi"})
X_data = pd.concat([X_withstr, df_oxygendummies], axis = "columns").drop(["mo1"], axis = "columns")

In [ ]:
print(len(X_data))

In [ ]:
allmat = set(X_data['ma1'].unique().tolist() + X_data['mb1'].unique().tolist() + X_data['mc1'].unique().tolist() + X_data['md1'].unique().tolist() + X_data['me1'].unique().tolist() + X_data['mf1'].unique().tolist() + X_data['mg1'].unique().tolist() + X_data['mh1'].unique().tolist() + X_data['mi1'].unique().tolist() + X_data['mj1'].unique().tolist())

In [ ]:
allmateriallist = list(allmat)

print(allmateriallist)

In [ ]:
"""del allmateriallist[0]

print(allmateriallist)"""

In [ ]:
#print(len(allmateriallist))

allmateriallistについて再起動ごとに順序が入れ替わる(delに影響が出る)のでpklで保存

In [ ]:
### pickleで保存（書き出し）
"""with open('allmateriallist.pkl', mode='wb') as fo:
    pickle.dump(allmateriallist, fo)"""

In [ ]:
### pickleで保存したファイルを読み込み
with open('allmateriallist.pkl', mode='br') as fi:
    allmateriallist = pickle.load(fi)

In [ ]:
print(allmateriallist)

In [ ]:
test = {'Nd':1, 'Br':2, 'C':3, 'No':4, 'U':5, 'Te':6, 'H':7, 'Np':8, 'Ho':9, 'Al':10, 'In':11, 'Mo':12, 'O':13, 'Cl':14, 'As':15, 'Ta':16, 'M':17, 'Fe':18, 'Yb':19, 'Ca':20, 'Y':21, 'Th':22, 'La':23, 'Sn':24, 'Hf':25, 'P':26, 'S ':27, 'Ce':28, 'Na':29, 'Li':30, 'Ag':31, 'Mn':32, 'Zr':33, 'Rh':34, 'B':35, 'Tl':36, 'Sm':37, 'Be':38, 'Cu':39, 'S':27, 'Sc':28, 'V':29, 'Bi':30, 'Ga':31, 'F':32, 'Er':33, 'Ni':34, 'D':35, 'Zn':36, 'Tb':37, 'Py':38, 'Cs':39, 'Os':40, 'Hg':41, 'Ru':42, 'Pt':43, 'I':44, 'Pr':45, 'Au':46, 'Ir':47, 'Tm':48, 'Re':49, 'Co':50, 'Nb':51, 'N':52, 'Am':53, 'Mg':54, 'Pc':55, 'Tc':56, 'Se':57, 'Ba':58, 'Ti':59, 'Cd':60, 'Gd':61, 'LI':30, 'Pu':63, 'Pd':64, 'W':65, 'Rb':66, 'Pb':67, 'Si':68, 'Ge':69, 'Lu':70, 'Cr':71, 'Y ':21, 'Sb':73, 'Sr':74, 'Dy':75, 'Eu':76, 'K':77}

In [ ]:
#上データはM、Y ,S ,LIを含んでいる

In [ ]:
testkeys = list(test.keys())
print(len(testkeys))

In [ ]:
dataindexamount = list(range(len(X_data)))

materialsdf = pd.DataFrame(columns = allmateriallist, index = dataindexamount)

In [ ]:
X_data = pd.concat([X_data, materialsdf], axis = "columns")

display(X_data)

In [ ]:
m1df = 10
for i in range(len(allmateriallist)): #各原子種類列インデックスごと
    for j in X_data.index: #各原子種類列インデックスごとの中の各行について
        for k in range(m1df): #各原子種類列インデックスごとの中の各行について、ma1、mb1と順々に処理
            if X_data.iloc[j, 2*k] == allmateriallist[i]: #ma1,mb1...ごとに書かれている原子名と一致する原子列において
                X_data.iloc[j, 22+i] = X_data.iloc[j, (2*k+1)] #上記の一致列に具体的な原子の数を代入する
        

display(X_data)

In [ ]:
X_data.to_pickle("X_data.pkl")

In [ ]:
X_data = pd.read_pickle("X_data.pkl")

In [ ]:
#用いるのはX_dataのname of structure以降のdataをXinput、y_tcshapedをyinput

In [ ]:
Xinput = X_data.loc[:, "name of structure":]
display(Xinput)

In [ ]:
alldata = pd.concat([Xinput, y["tc"]], axis = "columns")
alldata.to_pickle("alldata.pkl")

In [ ]:
alldata = pd.read_pickle("alldata.pkl")

# LightGBM回帰器のインスタンスを作成
model = lgb.LGBMRegressor()

params = {
        "n_jobs" : "-1", 
        "random_state" : "1", 
        "extra_trees" : True
        }

model.set_params(**params)

#バギングの有無
model = BaggingRegressor(model, random_state=1, n_jobs=1)

# 評価指標の設定
score_funcs = ["neg_mean_squared_error", "neg_mean_absolute_error", "r2"]

X = alldata.iloc[:, :(len(alldata.columns)-1)]

for i in range(len(allmateriallist)):
    X[allmateriallist[i]] = X[allmateriallist[i]].astype(float)

y = alldata.loc[:, "tc"]

# 通常のCV
cv_indicies = list(KFold().split(alldata))

# OOS予測値を計算
def my_cross_val_predict(estimator, X, y=None, cv=None):
    y_pred = y.copy()
    y_pred[:] = np.nan
    for train_idx, val_idx in cv:
        estimator.fit(X[train_idx], y[train_idx])
        y_pred[val_idx] = estimator.predict(X[val_idx])
    return y_pred

alldata["tc_pred"] = None

alldata["tc_pred"] = my_cross_val_predict(model, X.values, y.values, cv=cv_indicies)

# 交差検証を実行
score = cross_validate(model, X, y, cv=cv_indicies, scoring=score_funcs, return_estimator=True)

# 平均二乗誤差の平均値を出力
print(score["test_neg_mean_squared_error"].mean())
print(score["test_neg_mean_absolute_error"].mean())
print(score["test_r2"].mean())

In [ ]:
# 散布図の作成
sns.lmplot(x="tc", y="tc_pred", data=alldata, markers = ".", height=6, aspect=1.5)

# 傾きの表示
plt.annotate("傾き          = {}".format(round(alldata["tc_pred"].corr(alldata["tc"]), 3)), xy=(0, 0), xytext=(20, 160), fontsize=24)
plt.annotate("R2スコア　    = {}".format(round(score["test_r2"].mean(), 3)), xy=(0,0), xytext=(20, 180), fontsize=24)
plt.annotate("平均絶対誤差　= {}".format(round(score["test_neg_mean_absolute_error"].mean(), 3)), xy=(0,0), xytext=(20, 200), fontsize=24)

plt.title("GBDT　全データ", fontsize=24)

# グラフの表示
plt.show()

In [ ]:
dousankabutsu = ((alldata["Oarinashi"] == 1) & (alldata["Cu"].notnull()))
tetsu = ((alldata["Fe"].notnull()) & ((alldata["Oarinashi"] != 1) | (alldata["Cu"].isnull())))
nokori = ~ (dousankabutsu | tetsu)

alldataCu = alldata.loc[dousankabutsu, :]#鉄系超伝導体を得るために、鉄がありかつ銅と酸素が一緒にないものを得る
alldataFe = alldata.loc[tetsu, :]
alldatawithoutCuFe = alldata.loc[nokori, :]
display(alldataFe)
display(alldataCu)
print(alldataCu["Oarinashi"].value_counts())
display(alldatawithoutCuFe)
print(len(alldataCu) + len(alldataFe) + len(alldatawithoutCuFe))

alldataCu.to_pickle("alldataCu.pkl")
alldataFe.to_pickle("alldataFe.pkl")
alldatawithoutCuFe.to_pickle("alldatawithoutCuFe.pkl")

In [ ]:
alldataCu = pd.read_pickle("alldataCu.pkl")

# LightGBM回帰器のインスタンスを作成
model = lgb.LGBMRegressor()

params = {
        "n_jobs" : "-1", 
        "random_state" : "1", 
        "extra_trees" : True
        }

model.set_params(**params)

#バギングの有無
model = BaggingRegressor(model, random_state=1, n_jobs=1)

# 評価指標の設定
score_funcs = ["neg_mean_squared_error", "neg_mean_absolute_error", "r2"]

X = alldataCu.iloc[:, :(len(alldataCu.columns)-1)]

for i in range(len(allmateriallist)):
    X[allmateriallist[i]] = X[allmateriallist[i]].astype(float)

y = alldataCu.loc[:, "tc"]

# 通常のCV
cv_indicies = list(KFold().split(alldataCu))

# OOS予測値を計算
def my_cross_val_predict(estimator, X, y=None, cv=None):
    y_pred = y.copy()
    y_pred[:] = np.nan
    for train_idx, val_idx in cv:
        estimator.fit(X[train_idx], y[train_idx])
        y_pred[val_idx] = estimator.predict(X[val_idx])
    return y_pred

alldataCu["tc_pred"] = None

alldataCu["tc_pred"] = my_cross_val_predict(model, X.values, y.values, cv=cv_indicies)

# 交差検証を実行
score = cross_validate(model, X, y, cv=cv_indicies, scoring=score_funcs, return_estimator=True)

#モデルの詳細構造を出力
#model.fit(X, y)
#lgb.plot_tree(booster = model, figsize = (24, 16))

# 平均二乗誤差の平均値を出力
print(score["test_neg_mean_squared_error"].mean())
print(score["test_neg_mean_absolute_error"].mean())
print(score["test_r2"].mean())

In [ ]:
# 散布図の作成
sns.lmplot(x="tc", y="tc_pred", data=alldataCu, markers = ".", height=6, aspect=1.5)

# 傾きの表示
plt.annotate("傾き          = {}".format(round(alldataCu["tc_pred"].corr(alldataCu["tc"]), 3)), xy=(0, 0), xytext=(140, 10), fontsize=24)
plt.annotate("R2スコア　    = {}".format(round(score["test_r2"].mean(), 3)), xy=(0,0), xytext=(140, 25), fontsize=24)
plt.annotate("平均絶対誤差　= {}".format(round(score["test_neg_mean_absolute_error"].mean(), 3)), xy=(0,0), xytext=(140, 40), fontsize=24)

plt.title("GBDT　銅酸化物系超伝導体", fontsize=24)

# グラフの表示
plt.show()

In [ ]:
alldataFe = pd.read_pickle("alldataFe.pkl")

# LightGBM回帰器のインスタンスを作成
model = lgb.LGBMRegressor()

params = {
        "n_jobs" : "-1", 
        "random_state" : "1", 
        "extra_trees" : True
        }

model.set_params(**params)

#バギングの有無
model = BaggingRegressor(model, random_state=1, n_jobs=1)

# 評価指標の設定
score_funcs = ["neg_mean_squared_error", "neg_mean_absolute_error", "r2"]

X = alldataFe.iloc[:, :(len(alldataFe.columns)-1)]

for i in range(len(allmateriallist)):
    X[allmateriallist[i]] = X[allmateriallist[i]].astype(float)

y = alldataFe.loc[:, "tc"]

# 通常のCV
cv_indicies = list(KFold().split(alldataFe))

# OOS予測値を計算
def my_cross_val_predict(estimator, X, y=None, cv=None):
    y_pred = y.copy()
    y_pred[:] = np.nan
    for train_idx, val_idx in cv:
        estimator.fit(X[train_idx], y[train_idx])
        y_pred[val_idx] = estimator.predict(X[val_idx])
    return y_pred

alldataFe["tc_pred"] = None

alldataFe["tc_pred"] = my_cross_val_predict(model, X.values, y.values, cv=cv_indicies)

# 交差検証を実行
score = cross_validate(model, X, y, cv=cv_indicies, scoring=score_funcs, return_estimator=True)

# 平均二乗誤差の平均値を出力
print(score["test_neg_mean_squared_error"].mean())
print(score["test_neg_mean_absolute_error"].mean())
print(score["test_r2"].mean())

In [ ]:
# 散布図の作成
sns.lmplot(x="tc", y="tc_pred", data=alldataFe, markers = ".", height=6, aspect=1.5, ci=None)

# 傾きの表示
plt.annotate("傾き          = {}".format(round(alldataFe["tc_pred"].corr(alldataFe["tc"]), 3)), xy=(0, 0), xytext=(30, 120), fontsize=24)
plt.annotate("R2スコア　    = {}".format(round(score["test_r2"].mean(), 3)), xy=(0,0), xytext=(30, 132), fontsize=24)
plt.annotate("平均絶対誤差　= {}".format(round(score["test_neg_mean_absolute_error"].mean(), 3)), xy=(0,0), xytext=(30, 144), fontsize=24)

plt.title("GBDT　鉄系超伝導体", fontsize=24)

# グラフの表示
plt.show()

In [ ]:
alldatawithoutCuFe = pd.read_pickle("alldatawithoutCuFe.pkl")

# LightGBM回帰器のインスタンスを作成
model = lgb.LGBMRegressor()

params = {
        "n_jobs" : "-1", 
        "random_state" : "1", 
        "extra_trees" : True
        }

model.set_params(**params)

#バギングの有無
model = BaggingRegressor(model, random_state=1, n_jobs=1)

# 評価指標の設定
score_funcs = ["neg_mean_squared_error", "neg_mean_absolute_error", "r2"]

X = alldatawithoutCuFe.iloc[:, :(len(alldatawithoutCuFe.columns)-1)]

for i in range(len(allmateriallist)):
    X[allmateriallist[i]] = X[allmateriallist[i]].astype(float)

y = alldatawithoutCuFe.loc[:, "tc"]

# 通常のCV
cv_indicies = list(KFold().split(alldatawithoutCuFe))

# OOS予測値を計算
def my_cross_val_predict(estimator, X, y=None, cv=None):
    y_pred = y.copy()
    y_pred[:] = np.nan
    for train_idx, val_idx in cv:
        estimator.fit(X[train_idx], y[train_idx])
        y_pred[val_idx] = estimator.predict(X[val_idx])
    return y_pred

alldatawithoutCuFe["tc_pred"] = None

alldatawithoutCuFe["tc_pred"] = my_cross_val_predict(model, X.values, y.values, cv=cv_indicies)

# 交差検証を実行
score = cross_validate(model, X, y, cv=cv_indicies, scoring=score_funcs, return_estimator=True)

# 平均二乗誤差の平均値を出力
print(score["test_neg_mean_squared_error"].mean())
print(score["test_neg_mean_absolute_error"].mean())
print(score["test_r2"].mean())

In [ ]:
# 散布図の作成
sns.lmplot(x="tc", y="tc_pred", data=alldatawithoutCuFe, markers = ".", height=6, aspect=1.5, ci=None)

# 傾きの表示
plt.annotate("傾き          = {}".format(round(alldatawithoutCuFe["tc_pred"].corr(alldatawithoutCuFe["tc"]), 3)), xy=(0, 0), xytext=(10, 100), fontsize=24)
plt.annotate("R2スコア　    = {}".format(round(score["test_r2"].mean(), 3)), xy=(0,0), xytext=(10, 112), fontsize=24)
plt.annotate("平均絶対誤差　= {}".format(round(score["test_neg_mean_absolute_error"].mean(), 3)), xy=(0,0), xytext=(10, 124), fontsize=24)

plt.title("GBDT　銅酸化物系・鉄系以外", fontsize=24)

# グラフの表示
plt.show()